# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [2]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

API key looks good so far


In [4]:
# set up environment

question_system_prompt = "You are provided with a technical question about AI and LLMs \
Provide the most accurate answer and explain in detail to a student of LLM engineering the answer \n"


In [15]:
# here is the question; type over this to ask something new

question = """
how do i resolve this issue:
remote: Permission to renzvalentino/llm_engineering.git denied to samesquejo.
fatal: unable to access 'https://github.com/renzvalentino/llm_engineering.git/': The requested URL returned error: 403
"""

In [16]:
# Get gpt-4o-mini to answer, with streaming

def stream_question_answer(question):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": question_system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question_answer(question)

The issue you're encountering is related to GitHub permissions. Let's break down the error message and provide you with a step-by-step approach to resolving it.

### Error Breakdown:
- **"remote: Permission to renzvalentino/llm_engineering.git denied to samesquejo."**
  - This indicates that the user `samesquejo` does not have the necessary permissions to push to the repository `renzvalentino/llm_engineering.git`.

- **"fatal: unable to access 'https://github.com/renzvalentino/llm_engineering.git/': The requested URL returned error: 403"**
  - The `403` HTTP status code means "Forbidden", which confirms that your authentication to access the repository is failing due to permission issues.

### Steps to Resolve the Issue:

1. **Check Your Role:**
   - First, confirm whether you are a contributor to the repository. Only users with specific roles (like Admin, Write, or Maintainer) can push changes. If you are only a Read-Only collaborator or not listed as a collaborator at all, you will need to request access from the repo owner.

2. **Check Authentication Method:**
   - If you are using HTTPS for pushing, ensure you have the correct credentials set up. You may need to configure your personal access token if your account security settings have changed. GitHub no longer supports password-based authentication for HTTPS. You can generate a personal access token by:
     - Going to GitHub and logging in.
     - Navigate to **Settings -> Developer settings -> Personal access tokens**.
     - Click on “Generate new token” and grant it appropriate scopes (like repo access).
     - Use this token in place of your password when prompted by Git.

3. **Caching / Credential Helper:**
   - If you previously authenticated with incorrect credentials, Git may be caching those. To clear cached credentials, you can use:
     bash
     git credential-cache exit
     
   - Alternatively, you can find the stored credentials on your OS.
     - On Windows: Credential Manager.
     - On macOS: Keychain Access.
     - On Linux: Check `~/.git-credentials`.

4. **Verify the Remote URL:**
   - Double-check the remote repository URL you are trying to push to. You can verify by running:
     bash
     git remote -v
     
   - Make sure that the URL matches exactly the repository you intend to push to. If it doesn't, you can update it using:
     bash
     git remote set-url origin https://github.com/renzvalentino/llm_engineering.git
     

5. **Check Repository Settings:**
   - If you're the owner or a collaborator, ensure that the repository settings allow for your user access and check branch protection rules that might restrict access to certain branches.

6. **Log Out and Log Back In:**
   - Sometimes refreshing your authentication state can help. Log out from GitHub and reauthorize.

### Conclusion:
If you've verified all these steps and still face issues, reach out to the repository owner to ensure that you have correct permissions or to see if there are any additional guidelines they wish you to follow. Once the permission checks are cleared and your credentials are properly established, you should be able to push to the repository without encountering the error.

In [11]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [17]:
# Get Llama 3.2 to answer

def stream_question_answer_local(question):
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama_via_openai.chat.completions.create(
        model=MODEL_LLAMA,
        messages=[
            {"role": "system", "content": question_system_prompt},
            {"role": "user", "content": question}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

stream_question_answer_local(question)

**Resolving the 'Permission Denied' Issue on GitHub**

As a student of LLM engineering, you're likely working with open-source projects and collaborating with others. In this case, it seems like you've encountered an issue when trying to clone the `llm_engineering` repository from GitHub.

The error message indicates that GitHub has denied your request due to permission issues. This is usually caused by one of two reasons:

1. **Repository permissions**: The owner of the repository (`renzvalentino`) might have restricted access controls, denying you permission to clone the repository.
2. **API rate limiting or blocking**: Your account's API credentials may have been flagged for excessive requests, causing GitHub to block your attempts to access certain repositories.

To resolve this issue, follow these steps:

**Option 1: Find an alternate repository owner with permission**

Try cloning a different repository from `renzvalentino` that has been forked by someone with permissions. This way, you can still work with the desired project structure and content while ensuring you have the necessary access.

1. Search for similar repositories on GitHub using keywords like `llm-engineering`, `LLM development`, or other relevant terms.
2. Filter the results to show only open-source projects related to LLM engineering.
3. Look for a repository with an owner who has been forked from `renzvalentino` or has contributed to the original project.
4. Clone the new repository, and you should have access to its content.

**Option 2: Obtain permission from the repository owner**

If you're interested in working with the official `llm_engineering` repository, try resolving the issue by obtaining permission from the owner (`renzvalentino`). You can do this by sending an email or direct message to them explaining your situation and asking for permission.

1. Find the repository's homepage on GitHub (<https://github.com/renzvalentino/llm_engineering.git/>).
2. Click on **Issues** or **Pull requests**, depending on whether you want to request a review or contribute to the main codebase.
3. In the issue description, explain your background and why you're interested in working with the repository. Provide your educational institution, course materials, or relevant projects related to LLM engineering that demonstrate your expertise.
4. Request permission from `renzvalentino` by stating that you're willing to provide context, follow their guidelines, and respect their project's ownership.

Example email template:

Subject: ReQuesting Permission for llm_engineering Repository

Dear RenzValentino,

I'm [Your Name], a [Year] student at [University Name]. I've been studying [Course Name] as part of my LLM engineering studies. While working on [Relevant Project Materials], I stumbled upon your repository (llm_engineering) and thought it would be an excellent learning resource.

However, when trying to clone the repository, I encountered a 403 Forbidden error due to permission issues. After researching online, I realized that access was restricted. As a result, I'd like to request permission to work with the llm_engineering project under your ownership.

If granted permission, I'll ensure to provide context for my contributions and respect the original project's ownership. Here are some highlights from my background:

* **Projects:**
	+ [Project 1]: Developed an LLM-based chatbot using Python (2022)
	+ [Project 2]: Improvised a transformer model using TensorFlow (2023)

**What would I like to achieve with llm_engineering?**

* Gain hands-on experience with the latest tools and techniques in LLM engineering.
* Develop collaboration opportunities with professionals within the field.

Please let me know if this is something you're open to discussing further, and we can go from there. If approved, please provide detailed guidelines for contributing to the project, including any necessary access or setup information.

Thank you for your time and consideration!

Best regards,

[Your Name]

**Option 3: Request an API key or GitHub Developer account**

As a student developer, requesting permission might take some time. Alternatively, consider establishing yourself as a trusted contributor by registering for a **GitHub Developer account**

1. Go to the [GitHub Developer account page](https://developer.github.com).
2. Sign up for an organization (free) account.
3. Claim your GitHub profile and fill out all fields requested in the application process.
4. Fill out the "GitHub Profile" section by verifying your identity through an email address, social media credentials or other trusted authentication sources.

Create a new personal token to use it to access API endpoints on the main repository page or you can also add it in the settings or profiles in GitHub repositories.

**After resolving permission issues:**

* Check that your GitHub account has adequate access permissions. This may require verifying that your email address and user ID match your account information.
* Look up instructions for using your **GitHub tokens**, like those created above, depending on which version of the new API you prefer (1 or 2).

When you've addressed these steps, try cloning the repository again, ensuring you follow any specific guidelines provided by `renzvalentino` or other collaborators. If you're still encountering issues, it may be a sign that access restrictions are in place due to malicious activity.

To avoid future permissions issues when contributing to projects by establishing yourself as a trusted and engaged developer in open-source initiatives: